In [1]:
d = {"key":"val", "key1":"val1"}

In [2]:
d['key']

'val'

In [4]:
from box import ConfigBox

In [5]:
d2 = ConfigBox({"key":"val", "key1":"val1"})

In [8]:
d2.key1
print(type(d), type(d2))

<class 'dict'> <class 'box.config_box.ConfigBox'>


In [19]:
!pip install ensure

In [26]:
from ensure import ensure_annotations

In [27]:
# ensure annotations
@ensure_annotations
def get_product(x:int, y:int) -> int:
    return x*y

In [29]:
get_product(x=2, y=9)

18

In [30]:
import gdown

In [31]:
url="https://drive.google.com/file/d/1FvQL9V5QXtNJtFreSb21m2B1DQWECMt2/view?usp=sharing"

In [34]:
file_id=url.split("/")[-2]
file_id

'1FvQL9V5QXtNJtFreSb21m2B1DQWECMt2'

In [35]:
prefix ="https://drive.google.com/uc?export=download&id="
gdown.download(prefix+file_id, "kidneydisease.zip")

Downloading...
From (original): https://drive.google.com/uc?export=download&id=1FvQL9V5QXtNJtFreSb21m2B1DQWECMt2
From (redirected): https://drive.google.com/uc?export=download&id=1FvQL9V5QXtNJtFreSb21m2B1DQWECMt2&confirm=t&uuid=8cf55309-9b61-433d-b3f5-d0860c3a81a6
To: /Users/abcom/Documents/workingArea/pythonProjects/KidneyDisease/KidneyDiseaseClassification/research/kidneydisease.zip
100%|██████████| 943M/943M [01:16<00:00, 12.3MB/s] 


'kidneydisease.zip'

In [ ]:
import tensorflow as tf
from pathlib import Path

class Training:
    def __init__(self, config):
        self.config = config
        self.model = None  # Initialize model

    def get_base_model(self):
        # Load the base model
        self.model = tf.keras.models.load_model(
            self.config.updated_base_model_path
        )
        # Create a new optimizer instance for the loaded model
        self.optimizer = tf.keras.optimizers.Adam()  # Choose your optimizer

        # Compile the model with the new optimizer
        self.model.compile(optimizer=self.optimizer, 
                           loss='categorical_crossentropy',  # Adjust according to your needs
                           metrics=['accuracy'])

    def train_valid_generator(self):
        datagenerator_kwargs = dict(
            rescale=1.0 / 255,
            validation_split=0.20
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

        if self.config.params_is_augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                horizontal_flip=True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                **datagenerator_kwargs
            )
        else:
            train_datagenerator = valid_datagenerator

        self.train_generator = train_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="training",
            shuffle=True,
            **dataflow_kwargs
        )

    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)

    def train(self):
        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size

        self.model.fit(
            self.train_generator,
            epochs=self.config.params_epochs,
            steps_per_epoch=self.steps_per_epoch,
            validation_steps=self.validation_steps,
            validation_data=self.valid_generator
        )

        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )